In [ ]:
!git clone https://github.com/geohot/tinygrad.git

In [ ]:
cd tinygrad/

In [ ]:
!python3 -m pip install -e .

In [ ]:
pwd

In [1]:
import time, sys
import json
import random
from PIL import Image
import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import optim, BatchNorm2d
import tinygrad.nn as nn
from tinygrad.helpers import flatten
from tinygrad.nn.optim import SGD, Adam
from sklearn.datasets import fetch_openml
from tinygrad.state import safe_save, safe_load, get_state_dict, load_state_dict, torch_load

In [ ]:
# !pip install scikit-learn

In [ ]:
cd ../

In [2]:
dataset_dir = 'Dataset'

images = []
labels = []

# Iterate through the cat images
cat_dir = os.path.join(dataset_dir, 'Cat')
for filename in os.listdir(cat_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(cat_dir, filename)
        image = Image.open(image_path).convert('RGB')  
        image = image.resize((256, 256))
        images.append(np.array(image)/255)
        labels.append(0)

In [3]:
dog_dir = os.path.join(dataset_dir, 'Dog')
for filename in os.listdir(dog_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(dog_dir, filename)
        image = Image.open(image_path).convert('RGB')
        image = image.resize((256, 256))
        images.append(np.array(image)/255)
        labels.append(1)

: 

: 

In [14]:
data = list(zip(images, labels))
random.shuffle(data)

shuffled_images, shuffled_labels = zip(*data)

shuffled_images = np.array(shuffled_images, dtype=np.float64)
shuffled_labels = np.array(shuffled_labels)

In [15]:
shuffled_images.shape, shuffled_labels.shape

((24998, 224, 224), (24998,))

In [16]:
class CustomTinyNet:
    def __init__(self):
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)

    def forward(self, x):
        # print("Before Reshape: ", x.shape)
        x = x.reshape((x.shape[0], 3, 256, 256))
        # print("Before Conv-1: ", x.shape)
        x = self.conv1(x)
        x = x.leakyrelu()
        # print("After Conv-1+relu: ", x.shape)
        x = x.max_pool2d(2)  
        # print("After Pooling-1: ", x.shape)
        

        x = self.conv2(x)
        x = x.leakyrelu()
        # print("After Conv-2+relu: ", x.shape)
        x = x.max_pool2d(2) 
        # print("After Pooling-2: ", x.shape) 
       

        x = self.conv3(x)
        x = x.leakyrelu()
        # print("After Conv-3+relu: ", x.shape)
        x = x.max_pool2d(2)  
        # print("After Pooling-3: ", x.shape)
       

        x = x.reshape(x.shape[0], -1)
        # print("After Flatten: ", x.shape)

        x = self.fc1(x) 
        x = x.leakyrelu()
        # print("After FC-1: ", x.shape)

        x = self.fc2(x) 
        x = x.leakyrelu()
        # print("After FC-2: ", x.shape)

        x = self.fc3(x)
        # print("After FC-3: ", x.shape) 
        return x.log_softmax()

net = CustomTinyNet()

In [17]:
Tensor.training = True

In [18]:
def categorical_crossentropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten().astype(np.int32)
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  # correct loss for NLL, torch NLL loss returns one per row
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = Tensor(y)
  return out.mul(y).mean()

In [20]:
opt = Adam([net.conv1.weight, net.conv2.weight, net.conv3.weight], lr=0.01)

In [21]:
num_epochs = 2500

running_loss, correct, total = 0.0, 0.0, 0.0
start_time = time.time()

file_write = open("output.txt", "a")

for epoch in range(num_epochs):
    samp = np.random.randint(0, shuffled_images.shape[0], size=(128))
    batch = Tensor(shuffled_images[samp].astype(np.float32), requires_grad=False)
    labels_batch = shuffled_labels[samp]
    
    num_zeros = list(labels_batch).count(0)
    num_ones = list(labels_batch).count(1)
    # print(f"Zeros: {num_zeros}\tOnes: {num_ones}")
    

    out = net.forward(batch)

    loss = categorical_crossentropy(out, labels_batch)
    
    opt.zero_grad()

    loss.backward()
    

    pred = np.argmax(out.numpy().data, axis=-1)

    accuracies = [1 if prediction == actual_label else 0 for prediction, actual_label in zip(pred, labels_batch)]
    mean_accuracy = sum(accuracies) / len(accuracies)


    if epoch % 100 == 0:
        print(f"Time Taken: {time.time()-start_time:.1f}s, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.numpy():.5f}, Accuracy: {mean_accuracy:.5f}")
        
        file_write.write(f"Epochs: {epoch+1}/{num_epochs}\tLoss: {loss.numpy():.5f}\tAccuracy: {mean_accuracy:.5f}\tActual-label: {labels_batch}\tPrediction: {pred}\n")
        start_time = time.time()
    

file_write.close()


Time Taken: 1.8s, Epoch [1/2500], Loss: 0.69440, Accuracy: 0.40000
Time Taken: 19.9s, Epoch [101/2500], Loss: 0.69202, Accuracy: 0.58000
Time Taken: 16.7s, Epoch [201/2500], Loss: 0.69342, Accuracy: 0.47000
Time Taken: 17.4s, Epoch [301/2500], Loss: 0.69304, Accuracy: 0.51000
Time Taken: 16.8s, Epoch [401/2500], Loss: 0.69204, Accuracy: 0.58000
Time Taken: 18.7s, Epoch [501/2500], Loss: 0.69369, Accuracy: 0.45000
Time Taken: 16.7s, Epoch [601/2500], Loss: 0.69329, Accuracy: 0.49000
Time Taken: 21.8s, Epoch [701/2500], Loss: 0.69417, Accuracy: 0.42000
Time Taken: 19.4s, Epoch [801/2500], Loss: 0.69284, Accuracy: 0.52000
Time Taken: 19.5s, Epoch [901/2500], Loss: 0.69226, Accuracy: 0.57000
Time Taken: 17.5s, Epoch [1001/2500], Loss: 0.69340, Accuracy: 0.47000
Time Taken: 17.5s, Epoch [1101/2500], Loss: 0.69301, Accuracy: 0.51000
Time Taken: 17.6s, Epoch [1201/2500], Loss: 0.69341, Accuracy: 0.47000
Time Taken: 17.8s, Epoch [1301/2500], Loss: 0.69421, Accuracy: 0.41000
Time Taken: 18.9s, 

In [22]:
# set training flag to false
Tensor.training = False

st = time.perf_counter()
avg_acc = 0
for step in range(2000):
  # random sample a batch
  samp = np.random.randint(0, shuffled_images.shape[0], size=(128))
  batch = Tensor(shuffled_images[samp].astype('float32'), requires_grad=False)
  
  # get the corresponding labels
  batch_labels = shuffled_labels[samp]

  # forward pass
  out = net.forward(batch)

  # calculate accuracy
  pred = np.argmax(out.numpy(), axis=-1)

  # labels = [label for label in batch_labels]
  accuracies = [1 if prediction == actual_label else 0 for prediction, actual_label in zip(pred, labels_batch)]
  mean_accuracy = sum(accuracies) / len(accuracies)

print(f"Test Accuracy: {mean_accuracy}")
print(f"Time Taken To Test: {time.perf_counter() - st}")

Test Accuracy: 0.51
Time Taken To Test: 249.4559848000008


In [5]:
# first we need the state dict of our model
state_dict = get_state_dict(net)

# then we can just save it to a file
safe_save(state_dict, "model/model.safetensors")

In [8]:
import tinygrad.tensor as tensor
from PIL import Image
# Step 1: Preprocess the image
image = Image.open('Dataset/Cat/0.jpg').convert('L')
image = image.resize((256, 256))

# Step 2: Load the trained model
state_dict = safe_load("model/model.safetensors")

checkpoints = {
    'conv1.weight': state_dict['conv1.weight'],
    'conv1.bias': state_dict['conv1.bias'],
    'conv2.weight': state_dict['conv2.weight'],
    'conv2.bias': state_dict['conv2.bias'],
    'conv3.weight': state_dict['conv3.weight'],
    'conv3.bias': state_dict['conv3.bias'],
    'fc1.weight': state_dict['fc1.weight'],
    'fc1.bias': state_dict['fc1.bias'],
    'fc2.weight': state_dict['fc2.weight'],
    'fc2.bias': state_dict['fc2.bias'],
    'fc3.weight': state_dict['fc3.weight'],
    'fc3.bias': state_dict['fc3.bias']
}

model = CustomTinyNet()

load_state_dict(model, checkpoints)


image = np.array(image)

image = image.astype(np.float32) / 255.0  

start_time = time.time()
preprocessed_image = image.reshape((1, 3, 256, 256))

# Step 3: Forward pass
input_tensor = tensor.Tensor(preprocessed_image)  
output_tensor = net.forward(input_tensor)  

print(f"\n\nTime Taken to Predict the class: {time.time() - start_time: .4f}")

# Step 4: Obtain predictions
predicted_class= np.argmax(output_tensor.numpy(), axis=-1)
print("Predicted class:", predicted_class)

ram used:  0.01 GB, fc3.bias                                          : 100%|██████████| 12/12 [00:00<00:00, 336.56it/s]

loaded weights in 39.36 ms, 0.01 GB loaded at 0.25 GB/s


Time Taken to Predict the class:  0.0174
Predicted class: [1]
